In [1]:
import pandas as pd
import os
import datetime
import random
import glob
import paramiko
import numpy as np
import gc
from shutil import copyfile
os.getcwd()

'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3'

In [2]:
def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)

In [3]:
df_regular=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/output_2019-07-20/dfrfm_final_details_wemail_ziplabel_StoreQuad.csv",dtype=str)
df_18_plus=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/output_2019-07-20/df_ids_email_18_plus_shoppers.csv",dtype=str)

print(df_regular.shape)
print(df_18_plus.shape)


(18936098, 15)
(7337914, 3)


In [4]:
df_18_plus[pd.isnull(df_18_plus['customer_zip_code'])].shape

(6421320, 3)

In [5]:
df_18_plus[pd.notnull(df_18_plus['customer_zip_code'])].shape

(916594, 3)

In [6]:
samplerows=None

dfiddetail = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/combined_masterids_up_to_20181229_JL.csv',nrows = samplerows)
dfiddetail = dfiddetail.drop_duplicates('customer_id_hashed')
###
master_file_2018Q1_2019Q1=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
dfiddetail=master_file_2018Q1_2019Q1.append(dfiddetail).drop_duplicates("customer_id_hashed")
del master_file_2018Q1_2019Q1
###
master_file_gap=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
dfiddetail=master_file_gap.append(dfiddetail).drop_duplicates("customer_id_hashed")
del master_file_gap
###

all_master_files_after_201906=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
all_master_files_after_201906=[x for x in all_master_files_after_201906 if ("aster" in x) & (".txt" in x)]
all_master_files_after_201906=[x for x in all_master_files_after_201906 if x.split("ts/2019_by_weeks/MediaStorm_")[1][:10]>"2019-06-01"]
df_master_all_later_after_201906=pd.DataFrame()
for file in all_master_files_after_201906:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
    df_master_all_later_after_201906=df.append(df_master_all_later_after_201906)

dfiddetail=df_master_all_later_after_201906.append(dfiddetail).drop_duplicates("customer_id_hashed")
del df_master_all_later_after_201906

dfiddetail = dfiddetail.drop_duplicates('email_address_hash')

#dropped both email & id duplicated
###
gc.collect()
print("dfiddetail.shape",dfiddetail.shape)


dfiddetail['customer_zip_code'] = dfiddetail['customer_zip_code'].astype('str')
dfiddetail['customer_zip_code'] = dfiddetail['customer_zip_code'].str[0:5]
dfiddetail['customer_zip_code'].fillna('00000',inplace = True)
dfiddetail['customer_zip_code'] = dfiddetail['customer_zip_code'].apply(lambda x:x.zfill(5))


dfiddetail.head(2)

dfiddetail.shape (28200021, 3)


,customer_id_hashed,email_address_hash,customer_zip_code
0,767036eb85de7854fcdebc8c551e1132dc740d28d2ed8e...,058b39c75d316fb516d78b1b54d01dee298d20c5290ba0...,00nan
1,c5a36941a24a7470a30278f3491f0d525a0a202463d1aa...,ea6c5c7931fd759624c3f7756e89ff20aad37c33942bf1...,28043


In [7]:
df_18_plus=df_18_plus[~df_18_plus['customer_id_hashed'].isin(df_regular['customer_id_hashed'])]
df_18_plus=df_18_plus[~df_18_plus['email_address_hash'].isin(df_regular['email_address_hash'])]
print(df_18_plus.shape)

(7337563, 3)


In [8]:
df_regular=df_regular.drop_duplicates("customer_id_hashed").drop_duplicates("email_address_hash")
print(df_regular.shape)

df_18_plus=df_18_plus.drop_duplicates("customer_id_hashed").drop_duplicates("email_address_hash")
print(df_18_plus.shape)

(18936098, 15)
(7337563, 3)


In [9]:
del df_18_plus['customer_zip_code']
gc.collect()
print("df_18_plus.shape",df_18_plus.shape)
dfiddetail_zips=dfiddetail[['customer_id_hashed','customer_zip_code']].drop_duplicates("customer_id_hashed")
df_18_plus=pd.merge(df_18_plus,dfiddetail_zips,on="customer_id_hashed",how="left")
df_18_plus.shape

df_18_plus.shape (7337563, 2)


(7337563, 3)

In [10]:
df_18_plus[pd.isnull(df_18_plus['customer_zip_code'])].shape

(929370, 3)

In [11]:
df_18_plus[pd.notnull(df_18_plus['customer_zip_code'])].shape

(6408193, 3)

In [12]:
zipmap=pd.read_excel('/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx',
                     dtype = 'str',sheetname="unique_zips_full_footprint")
zipmap=zipmap.rename(columns={"zip_type":"zipcodegroup","zip_cd":"customer_zip_code"})

zipmap['zipcodegroup']=zipmap['zipcodegroup'].replace("trans_P","P")
zipmap['zipcodegroup']=zipmap['zipcodegroup'].replace("trans_S","S")
zipmap['zipcodegroup']=zipmap['zipcodegroup'].replace("zips_10","S")
zipmap.head(2)

,zipcodegroup,customer_zip_code
0,P,75040
1,P,20743


In [13]:
df_18_plus=pd.merge(df_18_plus,zipmap,on="customer_zip_code",how="left")
df_18_plus['zipcodegroup']=df_18_plus['zipcodegroup'].fillna("T")

In [14]:
df_18_plus_PS=df_18_plus[df_18_plus['zipcodegroup'].isin(['P','S'])]
df_18_plus_T=df_18_plus[df_18_plus['zipcodegroup']=="T"]


In [15]:
df_18_plus_T.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code,zipcodegroup
3,3d5aa085305d877141840603e61b1a88e58ccb23090deb...,401c280280cd1d418e419b5ac3f81cae1709287ef476fe...,00nan,T
18,32a628542a5047341fc60b6e14878de6cf4b0cca38f01c...,e2eebd4447564dd57458676c19b4c6588a2a3430d554cb...,00nan,T


In [16]:
df_18_plus_PS.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/output_2019-07-20/df_18_plus_PS.csv",
                     index=False)
df_18_plus_T.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/output_2019-07-20/df_18_plus_T.csv",
                     index=False)

In [17]:
random.seed(1)

total_rows=len(df_regular)

df_test_all_regular=pd.DataFrame()
df_control_all_regular=pd.DataFrame()

i_counter=0

df_regular=df_regular[['segment_Decile','segment_2019Q3','customer_id_hashed','email_address_hash']]

for seg,group in df_regular.groupby(['segment_Decile']):
    random_list=random.sample(range(len(group)), int(np.round(len(group)/total_rows*500000)))

    group=group.reset_index()
    del group['index']
    group=group.reset_index()
    df_control=group[group['index'].isin(random_list)]
    df_test=group[~group['index'].isin(random_list)]
    
    df_control['segment_Decile']="C_"+df_control['segment_Decile']+"_RFM"
    df_test['segment_Decile']="T_"+df_test['segment_Decile']+"_RFM"
    
    df_control['segment_2019Q3']="C_"+df_control['segment_2019Q3']+"_RFM"
    df_test['segment_2019Q3']="T_"+df_test['segment_2019Q3']+"_RFM"
    df_test_all_regular=df_test_all_regular.append(df_test)
    df_control_all_regular=df_control_all_regular.append(df_control)
    i_counter+=1
    print(i_counter,datetime.datetime.now())
    

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

1 2019-07-22 17:54:59.855533
2 2019-07-22 17:55:00.387114
3 2019-07-22 17:55:00.986589
4 2019-07-22 17:55:01.649001
5 2019-07-22 17:55:02.480584
6 2019-07-22 17:55:03.348772
7 2019-07-22 17:55:04.259239
8 2019-07-22 17:55:05.219729
9 2019-07-22 17:55:06.236765
10 2019-07-22 17:55:07.304856
11 2019-07-22 17:55:08.200320
12 2019-07-22 17:55:09.103892
13 2019-07-22 17:55:10.007099
14 2019-07-22 17:55:10.940036
15 2019-07-22 17:55:11.895758
16 2019-07-22 17:55:12.879013
17 2019-07-22 17:55:13.904854
18 2019-07-22 17:55:14.960520
19 2019-07-22 17:55:16.058715
20 2019-07-22 17:55:17.177531
21 2019-07-22 17:55:18.219685
22 2019-07-22 17:55:19.278515
23 2019-07-22 17:55:20.345353
24 2019-07-22 17:55:21.438582
25 2019-07-22 17:55:22.536173
26 2019-07-22 17:55:23.669251
27 2019-07-22 17:55:24.818202
28 2019-07-22 17:55:25.993604
29 2019-07-22 17:55:27.184167
30 2019-07-22 17:55:28.394704
31 2019-07-22 17:55:29.703089
32 2019-07-22 17:55:31.041288
33 2019-07-22 17:55:32.406505
34 2019-07-22 17:55

In [19]:
import random
random.seed(2)
total_rows=len(df_18_plus_PS)
df_18_plus_PS['segment']="lapsed_18_48_PS"
del df_18_plus_PS['customer_zip_code']
random_list=random.sample(range(total_rows), 100000)

df_18_plus_PS=df_18_plus_PS.reset_index()
del df_18_plus_PS['index']
df_18_plus_PS=df_18_plus_PS.reset_index()

df_18_plus_PS_control=df_18_plus_PS[df_18_plus_PS['index'].isin(random_list)]
df_18_plus_PS_test=df_18_plus_PS[~df_18_plus_PS['index'].isin(random_list)]

df_18_plus_PS_control['segment']="C_"+df_18_plus_PS_control['segment']+"_RFM"
df_18_plus_PS_test['segment']="T_"+df_18_plus_PS_test['segment']+"_RFM"
    
print("df_18_plus_PS_control.shape",df_18_plus_PS_control.shape)
print("df_18_plus_PS_test.shape",df_18_plus_PS_test.shape)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


df_18_plus_PS_control.shape (100000, 6)
df_18_plus_PS_test.shape (5487169, 6)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [20]:
folder_write="/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/output_2019-07-20/Test_and_Control/"
try:
    os.stat(folder_write)
except:
    os.mkdir(folder_write)

In [21]:
df_test_all_regular.to_csv(folder_write+"df_test_all_regular.csv",index=False)
df_control_all_regular.to_csv(folder_write+"df_control_all_regular.csv",index=False)

df_18_plus_PS_test.to_csv(folder_write+"df_18_plus_PS_test.csv",index=False)
df_18_plus_PS_control.to_csv(folder_write+"df_18_plus_PS_control.csv",index=False)

folder_write_inner_decile = folder_write+'by_decile_group/'
try:
    os.stat(folder_write_inner_decile)
except:
    os.mkdir(folder_write_inner_decile)
    
folder_write_inner_HML = folder_write+'by_HML_group/'
try:
    os.stat(folder_write_inner_HML)
except:
    os.mkdir(folder_write_inner_HML)

In [22]:
# Decile
# Test
i_counter=0
for seg,group in df_test_all_regular.groupby(['segment_Decile']):
    group=group[['customer_id_hashed','email_address_hash','segment_Decile','segment_2019Q3']]
    group.to_csv(folder_write_inner_decile+seg.replace(" ","_")+".csv",index=False)
    i_counter+=1
    print(i_counter,seg,datetime.datetime.now())
    
# Decile
# Control    
i_counter=0
for seg,group in df_control_all_regular.groupby(['segment_Decile']):
    group=group[['customer_id_hashed','email_address_hash','segment_Decile','segment_2019Q3']]
    group.to_csv(folder_write_inner_decile+seg.replace(" ","_")+".csv",index=False)
    i_counter+=1
    print(i_counter,seg,datetime.datetime.now())

1 T_Quadrant III_P_D01_2019Q3_RFM 2019-07-22 18:02:41.496110
2 T_Quadrant III_P_D02_2019Q3_RFM 2019-07-22 18:02:43.787997
3 T_Quadrant III_P_D03_2019Q3_RFM 2019-07-22 18:02:46.170734
4 T_Quadrant III_P_D04_2019Q3_RFM 2019-07-22 18:02:48.377772
5 T_Quadrant III_P_D05_2019Q3_RFM 2019-07-22 18:02:50.728081
6 T_Quadrant III_P_D06_2019Q3_RFM 2019-07-22 18:02:52.854543
7 T_Quadrant III_P_D07_2019Q3_RFM 2019-07-22 18:02:55.102093
8 T_Quadrant III_P_D08_2019Q3_RFM 2019-07-22 18:02:57.259027
9 T_Quadrant III_P_D09_2019Q3_RFM 2019-07-22 18:02:59.622117
10 T_Quadrant III_P_D10_2019Q3_RFM 2019-07-22 18:03:01.953820
11 T_Quadrant III_S_D01_2019Q3_RFM 2019-07-22 18:03:02.640763
12 T_Quadrant III_S_D02_2019Q3_RFM 2019-07-22 18:03:03.487516
13 T_Quadrant III_S_D03_2019Q3_RFM 2019-07-22 18:03:04.316100
14 T_Quadrant III_S_D04_2019Q3_RFM 2019-07-22 18:03:05.242236
15 T_Quadrant III_S_D05_2019Q3_RFM 2019-07-22 18:03:06.142424
16 T_Quadrant III_S_D06_2019Q3_RFM 2019-07-22 18:03:07.067272
17 T_Quadrant III

16 C_Quadrant III_S_D06_2019Q3_RFM 2019-07-22 18:04:27.692346
17 C_Quadrant III_S_D07_2019Q3_RFM 2019-07-22 18:04:27.721358
18 C_Quadrant III_S_D08_2019Q3_RFM 2019-07-22 18:04:27.750278
19 C_Quadrant III_S_D09_2019Q3_RFM 2019-07-22 18:04:27.782751
20 C_Quadrant III_S_D10_2019Q3_RFM 2019-07-22 18:04:27.812755
21 C_Quadrant III_T_D01_2019Q3_RFM 2019-07-22 18:04:27.823343
22 C_Quadrant III_T_D02_2019Q3_RFM 2019-07-22 18:04:27.839280
23 C_Quadrant III_T_D03_2019Q3_RFM 2019-07-22 18:04:27.851339
24 C_Quadrant III_T_D04_2019Q3_RFM 2019-07-22 18:04:27.866268
25 C_Quadrant III_T_D05_2019Q3_RFM 2019-07-22 18:04:27.879931
26 C_Quadrant III_T_D06_2019Q3_RFM 2019-07-22 18:04:27.897737
27 C_Quadrant III_T_D07_2019Q3_RFM 2019-07-22 18:04:27.914058
28 C_Quadrant III_T_D08_2019Q3_RFM 2019-07-22 18:04:27.931694
29 C_Quadrant III_T_D09_2019Q3_RFM 2019-07-22 18:04:27.949391
30 C_Quadrant III_T_D10_2019Q3_RFM 2019-07-22 18:04:27.966289
31 C_Quadrant II_P_D01_2019Q3_RFM 2019-07-22 18:04:27.998942
32 C_Quad

In [23]:
# HML
# Test
i_counter=0
for seg,group in df_test_all_regular.groupby(['segment_2019Q3']):
    group=group[['customer_id_hashed','email_address_hash','segment_2019Q3']].rename(columns={"segment_2019Q3":"segment"})
    group.to_csv(folder_write_inner_HML+seg.replace(" ","_")+".csv",index=False)
    i_counter+=1
    print(i_counter,seg,datetime.datetime.now())
    
# HML
# Control
i_counter=0
for seg,group in df_control_all_regular.groupby(['segment_2019Q3']):
    group=group[['customer_id_hashed','email_address_hash','segment_2019Q3']].rename(columns={"segment_2019Q3":"segment"})
    group.to_csv(folder_write_inner_HML+seg.replace(" ","_")+".csv",index=False)
    i_counter+=1
    print(i_counter,seg,datetime.datetime.now())

1 T_Quadrant III_P_H_2019Q3_RFM 2019-07-22 18:04:42.863660
2 T_Quadrant III_P_L_2019Q3_RFM 2019-07-22 18:04:48.790984
3 T_Quadrant III_P_M_2019Q3_RFM 2019-07-22 18:04:54.678416
4 T_Quadrant III_S_H_2019Q3_RFM 2019-07-22 18:04:57.498402
5 T_Quadrant III_S_L_2019Q3_RFM 2019-07-22 18:05:00.062210
6 T_Quadrant III_S_M_2019Q3_RFM 2019-07-22 18:05:02.407421
7 T_Quadrant III_T_H_2019Q3_RFM 2019-07-22 18:05:03.740786
8 T_Quadrant III_T_L_2019Q3_RFM 2019-07-22 18:05:05.242665
9 T_Quadrant III_T_M_2019Q3_RFM 2019-07-22 18:05:06.485944
10 T_Quadrant II_P_H_2019Q3_RFM 2019-07-22 18:05:09.774946
11 T_Quadrant II_P_L_2019Q3_RFM 2019-07-22 18:05:12.025067
12 T_Quadrant II_P_M_2019Q3_RFM 2019-07-22 18:05:14.503094
13 T_Quadrant II_S_H_2019Q3_RFM 2019-07-22 18:05:15.785849
14 T_Quadrant II_S_L_2019Q3_RFM 2019-07-22 18:05:16.717545
15 T_Quadrant II_S_M_2019Q3_RFM 2019-07-22 18:05:17.637165
16 T_Quadrant II_T_H_2019Q3_RFM 2019-07-22 18:05:18.208373
17 T_Quadrant II_T_L_2019Q3_RFM 2019-07-22 18:05:18.6404

In [24]:
df_control_all_regular.shape

(500002, 5)

In [25]:
df_18_plus_PS_control.shape

(100000, 6)

# Summary

In [26]:
HML_files=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/output_2019-07-20/Test_and_Control/by_HML_group/*.csv")
HML_files=[x for x in HML_files if "_T_" not in x]
print(len(HML_files))


48


In [27]:
# summary_all_HML

df_all_HML=pd.DataFrame()
for file in glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/output_2019-07-20/Test_and_Control/by_HML_group/*.csv"):
    df=pd.read_csv(file,dtype=str)
    df_all_HML=df_all_HML.append(df)
summary_all_HML=df_all_HML.groupby("segment")['customer_id_hashed'].count().to_frame().reset_index()

del df_all_HML
gc.collect()

250

In [28]:
# summary_all_decile

df_all_decile=pd.DataFrame()
for file in glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/output_2019-07-20/Test_and_Control/by_decile_group/*.csv"):
    df=pd.read_csv(file,dtype=str)
    df_all_decile=df_all_decile.append(df)
summary_all_decile=df_all_decile.groupby("segment_Decile")['customer_id_hashed'].count().to_frame().reset_index()

del df_all_decile
gc.collect()

315

In [29]:
summary_all_HML.head(2)

,segment,customer_id_hashed
0,C_Quadrant III_P_H_2019Q3_RFM,42851
1,C_Quadrant III_P_L_2019Q3_RFM,31860


In [30]:
summary_all_decile=summary_all_decile.sort_values("segment_Decile")
summary_all_HML=summary_all_HML.sort_values("segment")


In [31]:
upload_folder="/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/upload_files/"
try:
    os.stat(upload_folder)
except:
    os.mkdir(upload_folder)

for local_file in HML_files:
    base_name=os.path.basename(local_file)
    remote_name=upload_folder+base_name
    copyfile(local_file,remote_name)

In [32]:
# summary 18_48_lapsed

df_18_49_lapsed_test_PS=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/output_2019-07-20/Test_and_Control/df_18_plus_PS_test.csv",
                            dtype=str,usecols=['customer_id_hashed','email_address_hash','segment'])
df_18_49_lapsed_control_PS=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/output_2019-07-20/Test_and_Control/df_18_plus_PS_control.csv",
                            dtype=str,usecols=['customer_id_hashed','email_address_hash','segment'])

print("df_18_49_lapsed_test_PS,",df_18_49_lapsed_test_PS.shape)
print("df_18_49_lapsed_control_PS,",df_18_49_lapsed_control_PS.shape)


df_18_49_lapsed_test_PS, (5487169, 3)
df_18_49_lapsed_control_PS, (100000, 3)


In [33]:
df_18_49_lapsed_test_PS=df_18_49_lapsed_test_PS[['customer_id_hashed','email_address_hash','segment']]
df_18_49_lapsed_control_PS=df_18_49_lapsed_control_PS[['customer_id_hashed','email_address_hash','segment']]


df_18_49_lapsed_test_PS.to_csv(upload_folder+"T_lapsed_18_48_PS_2019Q3_RFM.csv",index=False)
df_18_49_lapsed_control_PS.to_csv(upload_folder+"C_lapsed_18_48_PS_2019Q3_RFM.csv",index=False)


In [34]:
len_test_lapsed_18_49_PS_ids=df_18_49_lapsed_test_PS['customer_id_hashed'].nunique()
len_control_lapsed_18_49_PS_ids=df_18_49_lapsed_control_PS['customer_id_hashed'].nunique()

summary_lapsed=pd.DataFrame(columns=["segment","ids"])

df_test_lapsed_PS_count=pd.DataFrame({"segment":"test_lapsed_18_49_PS","ids":len_test_lapsed_18_49_PS_ids},index=[0])
df_control_lapsed_PS_count=pd.DataFrame({"segment":"control_lapsed_18_49_PS","ids":len_control_lapsed_18_49_PS_ids},index=[1])
summary_lapsed=summary_lapsed.append(df_test_lapsed_PS_count).append(df_control_lapsed_PS_count)

In [35]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/output_2019-07-20/BL_summary_2019_Q3_audience_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
summary_all_HML.to_excel(writer,"summary_all_HML",index=False)
summary_all_decile.to_excel(writer,"summary_all_decile",index=False)
summary_lapsed.to_excel(writer,"summary_lapsed",index=False)
writer.save()